In [1]:
import pandas as pd
import numpy as np


In [2]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("epl-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("epl-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("epl-2025.csv")

In [3]:
#za sezonu 25 6
df_25_6["Home Team"] = df_25_6["Home Team"].replace("Nott'm Forest", 'Nottingham Forest')
df_25_6["Away Team"] = df_25_6["Away Team"].replace("Nott'm Forest", 'Nottingham Forest')
#za sezonu 24 5
df_24_5["Home Team"] = df_24_5["Home Team"].replace("Nott'm Forest", 'Nottingham Forest')
df_24_5["Away Team"] = df_24_5["Away Team"].replace("Nott'm Forest", 'Nottingham Forest')

## Obavezno!!!

In [4]:
#odrediti duzinu sezone
df_25_6 = df_25_6.head(120)
df_25_6.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
115,120,12,22/11/2025 15:00,Molineux Stadium,Wolves,Crystal Palace,0 - 2
116,119,12,22/11/2025 17:30,St. James' Park,Newcastle,Man City,2 - 1
117,116,12,23/11/2025 14:00,Elland Road,Leeds,Aston Villa,1 - 2
118,111,12,23/11/2025 16:30,Emirates Stadium,Arsenal,Spurs,4 - 1
119,118,12,24/11/2025 20:00,Old Trafford,Man Utd,Everton,0 - 1


## Priprema Podataka i df-a

In [5]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

rez_dom = df_BL["Result"].str.slice(0,1)
rez_gost = df_BL["Result"].str.slice(4,5)

df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]

df1 = df_rezultatski

## Definisanje Funckije za Izracun Forme

In [6]:
def klub(tim):
    # Filtriramo DataFrame
    df = df1[(df1["H"] == tim) | (df1["A"] == tim)].copy()
    
    uslovi = [
    (df['H'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] > df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] > df["Rez2"]),]
    
    vrijednosti = [-1, 1, 0, 0, 1, -1]
    df['Forma'] = np.select(uslovi, vrijednosti, default=0)
    df = df.drop(["Rez1","Rez2"],axis=1).tail(5)
    rezultat = df["Forma"].sum() / len(df["Forma"])
    return rezultat

## Provjera Rada te Funckije

In [7]:
klub('Leeds')

-0.6

## Stvaranje Petlje za Automatski Izracun Forme Svih Klubova

In [8]:
#izvlacenje naziva svih klubova
timovi = pd.concat([df1["H"], df1["A"]]).unique()
timovi

array(['Burnley', 'Arsenal', 'Bournemouth', 'Brighton', 'Everton',
       'Sheffield Utd', 'Newcastle', 'Brentford', 'Chelsea', 'Man Utd',
       'Nottingham Forest', 'Fulham', 'Liverpool', 'Wolves', 'Spurs',
       'Man City', 'Aston Villa', 'West Ham', 'Crystal Palace', 'Luton',
       'Ipswich', 'Leicester', 'Southampton', 'Sunderland', 'Leeds'],
      dtype=object)

In [9]:
rezultati = {}

In [10]:
#petlja
for tim in timovi:
    rezultat = klub(tim)
    rezultati[tim] = rezultat

# Kreiranje DataFrame-a iz rezultata
df_forme = pd.DataFrame(list(rezultati.items()),columns = ["Klub","Rezultat"])
df_forme["Rezultat"] = df_forme["Rezultat"]

## Rezulat Forme za Sve

In [11]:
#Izracun
df_forme 

,Klub,Rezultat
0,Burnley,-0.2
1,Arsenal,0.8
2,Bournemouth,-0.2
3,Brighton,0.4
4,Everton,0.0
5,Sheffield Utd,-1.0
6,Newcastle,-0.2
7,Brentford,0.2
8,Chelsea,0.6
9,Man Utd,0.2


In [12]:
#Ponovna provjera vrijednosti...
klub("Brighton")

0.4

In [13]:
df_forme.to_csv("forma_h.csv")

In [14]:
df_forme["Rezultat"] = df_forme["Rezultat"] * -1

In [15]:
df_forme


,Klub,Rezultat
0,Burnley,0.2
1,Arsenal,-0.8
2,Bournemouth,0.2
3,Brighton,-0.4
4,Everton,-0.0
5,Sheffield Utd,1.0
6,Newcastle,0.2
7,Brentford,-0.2
8,Chelsea,-0.6
9,Man Utd,-0.2


In [16]:
df_forme.to_csv("forma_a.csv")